# Advent of Code

## 2024-012-012
## 2024 012

https://adventofcode.com/2024/day/12

In [1]:
import time
import sys

def neighbors(r, c, rows, cols):
    # Return valid orth neighbors
    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols:
            yield nr, nc

def main():
    start = time.perf_counter_ns()
    
    # Read input
    grid = []
    with open("input.txt", "r") as f:
        # The provided input seems to be very large and spread over multiple lines
        # We'll read all lines into a grid of characters
        # Each line in the puzzle input is presumably of the same length.
        for line in f:
            line = line.strip()
            if line:
                grid.append(list(line))
    
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False]*cols for _ in range(rows)]

    total_price = 0

    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                # Identify the region starting here
                plant_type = grid[i][j]
                stack = [(i, j)]
                visited[i][j] = True
                area = 0
                perimeter = 0

                while stack:
                    r, c = stack.pop()
                    area += 1
                    # Check all four sides for perimeter contributions
                    for nr, nc in [(r-1,c), (r+1,c), (r,c-1), (r,c+1)]:
                        if not (0 <= nr < rows and 0 <= nc < cols):
                            # Outside boundary contributes to perimeter
                            perimeter += 1
                        else:
                            if grid[nr][nc] != plant_type:
                                # Different type contributes to perimeter
                                perimeter += 1
                            else:
                                # Same type: part of same region
                                if not visited[nr][nc]:
                                    visited[nr][nc] = True
                                    stack.append((nr, nc))
                
                # Now we have area and perimeter for this region
                price = area * perimeter
                total_price += price

    end = time.perf_counter_ns()
    elapsed_ns = end - start
    elapsed_s = elapsed_ns / 1_000_000_000

    # Print total price and time with nanosecond precision
    # Example format: 1.234 567 890 s (just an example; we can just print the seconds with high precision)
    # We'll print a standard float with many decimals:
    print(total_price)
    print(f"{elapsed_s:.9f} s")

if __name__ == "__main__":
    main()

1522850
0.050787800 s


In [3]:
import time
import sys
from collections import defaultdict, deque

def main():
    start = time.perf_counter_ns()
    # Read input
    grid = []
    with open("sample-input-for-part-002-number-003.txt", "r") as f:
        for line in f:
            line = line.strip()
            if line:
                grid.append(list(line))
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False]*cols for _ in range(rows)]

    def neighbors(r, c):
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    total_price = 0

    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                # Identify region
                plant_type = grid[i][j]
                stack = [(i,j)]
                visited[i][j] = True
                region_cells = []
                while stack:
                    r, c = stack.pop()
                    region_cells.append((r,c))
                    for nr, nc in neighbors(r,c):
                        if not visited[nr][nc] and grid[nr][nc] == plant_type:
                            visited[nr][nc] = True
                            stack.append((nr,nc))

                # Compute area:
                area = len(region_cells)

                # Collect boundary edges
                # We'll represent edges by their endpoints in a consistent manner:
                # For a vertical edge between (r,c) and (r+1,c): use ((r,c),(r+1,c)) with smallest first.
                # For a horizontal edge between (r,c) and (r,c+1): use ((r,c),(r,c+1)).
                edges = []
                region_set = set(region_cells)
                for (r,c) in region_cells:
                    # Up edge
                    if r == 0 or grid[r-1][c] != plant_type:
                        edges.append(((r,c),(r,c+1)))  # horizontal top edge
                    # Down edge
                    if r == rows-1 or grid[r+1][c] != plant_type:
                        edges.append(((r+1,c),(r+1,c+1))) # horizontal bottom edge
                    # Left edge
                    if c == 0 or grid[r][c-1] != plant_type:
                        edges.append(((r,c),(r+1,c))) # vertical left edge
                    # Right edge
                    if c == cols-1 or grid[r][c+1] != plant_type:
                        edges.append(((r,c+1),(r+1,c+1))) # vertical right edge

                # Now we have all boundary edges. We must form loops and count sides.
                # Build graph: vertex is a point (r,c), edges connect two vertices that form a boundary edge.
                # Each boundary edge: (v1,v2) where v1,v2 are points.
                # We'll find cycles in this graph.
                
                # Adjacency list
                adj = defaultdict(list)
                for e in edges:
                    v1, v2 = e
                    adj[v1].append(v2)
                    adj[v2].append(v1)

                # Find loops (each vertex likely has even degree)
                # We'll find cycles by traversing edges until all are used.
                # A simple approach: for each connected component, do a DFS to find cycles.
                
                # To do this robustly, we must track visited edges, not just vertices.
                # Represent edges in a normalized form.
                edge_visited = set()

                def edge_key(u,v):
                    return (u,v) if u < v else (v,u)

                sides_count = 0

                # We'll iterate over all vertices in this boundary graph and find cycles
                # Each connected component may have multiple cycles if there are holes.
                # Actually, for orthogonal polygons, each connected component of edges should form one or more closed loops.
                # We'll do a DFS that always tries to follow edges until we return to the start.

                visited_vertices = set()

                for start_vertex in adj:
                    if start_vertex not in visited_vertices:
                        # Explore this component
                        # We'll find all cycles by doing a 'cycle finding' approach:
                        stack = [start_vertex]
                        parent = {start_vertex: None}
                        visited_vertices.add(start_vertex)
                        # For orth boundary, each component should form one or more cycles.
                        # However, each edge belongs to exactly one cycle in a rectilinear polygon boundary.
                        # Actually, we can do a simpler approach:
                        # Since it's a polygon boundary, each component is basically a set of cycles.
                        # We can extract cycles by using a Hierholzer-like approach (like Eulerian circuit),
                        # since all vertices should have even degree (2 or more edges), forming closed loops.

                        # We'll perform an Eulerian traversal to extract cycles:
                        # Convert adj to a structure of edges. Then find Eulerian cycles.
                        
                        # Copy edges of this component
                        comp_edges = []
                        comp_adj = defaultdict(list)

                        # Collect all edges in this component using a BFS/DFS
                        comp_vertices = set()
                        q = deque([start_vertex])
                        while q:
                            v = q.popleft()
                            if v in comp_vertices:
                                continue
                            comp_vertices.add(v)
                            for nx in adj[v]:
                                if nx not in comp_vertices:
                                    q.append(nx)

                        # Build subgraph for component
                        for v in comp_vertices:
                            for w in adj[v]:
                                if w in comp_vertices:
                                    ek = edge_key(v,w)
                                    comp_adj[v].append(w)

                        # Now find Eulerian cycles in this component.
                        # Every edge must be used exactly once. Graph is even degree, so Eulerian circuit(s).
                        # Hierholzer's algorithm:
                        def eulerian_cycles():
                            # Local copy of adjacency lists with removable edges
                            local_adj = {v: comp_adj[v][:] for v in comp_vertices}
                            # track usage of edges by removing them from local_adj
                            
                            # Each Eulerian cycle can be found by a stack approach
                            stack = []
                            cycles = []
                            # Start from any vertex
                            start_v = next(iter(comp_vertices))
                            stack = [start_v]
                            path = []
                            while stack:
                                u = stack[-1]
                                if local_adj[u]:
                                    w = local_adj[u].pop()
                                    stack.append(w)
                                else:
                                    path.append(stack.pop())
                            # 'path' now is an Eulerian tour (since we have a single connected component that should form closed loops of edges)
                            # But there might be multiple cycles if the shape has holes. Actually, Eulerian path returns one big cycle if strongly connected.
                            # For orth polygons with holes, all edges form a set of closed loops. Hierholzer would produce a single cycle if the graph is a single polygon.
                            # With holes, the boundary might appear as multiple cycles connected at vertices. But since each cycle is disjoint set of edges,
                            # Actually, all edges form a disjoint union of cycles. Hierholzer’s will find all of them combined in 'path' but with repeats of vertices where it splits off.
                            # The 'path' from Hierholzer's algorithm includes the start vertex repeated at end.
                            # It's a known approach: If we consider the trail from Hierholzer, each time we return to a vertex that still has edges, we form a cycle.

                            # The returned 'path' is an Eulerian path that visits all edges exactly once.
                            # To extract cycles:
                            # The path is something like: v0 ... v1 ... v0 ... v2 ... v0
                            # Every time we come back to the start vertex of a sub-tour, we completed a cycle.
                            # Actually, Hierholzer usually records cycles as it backtracks. Let's do the official approach:
                            
                            # Let's implement Hierholzer properly to get all cycles explicitly:
                            def hierholzer_subroutine(start):
                                cycle_stack = [start]
                                route = []
                                while cycle_stack:
                                    u = cycle_stack[-1]
                                    if local_adj[u]:
                                        w = local_adj[u].pop()
                                        cycle_stack.append(w)
                                    else:
                                        route.append(cycle_stack.pop())
                                return route[::-1]  # route in correct order

                            # Since we modified local_adj once, let's restore and properly do Hierholzer:
                            local_adj = {v: comp_adj[v][:] for v in comp_vertices}
                            # We'll do the standard Hierholzer: 
                            # Start from a vertex with edges
                            u = start_v
                            stack = [u]
                            path = []
                            # store intermediate cycles
                            while stack:
                                u = stack[-1]
                                if local_adj[u]:
                                    w = local_adj[u].pop()
                                    stack.append(w)
                                else:
                                    path.append(stack.pop())
                            path.reverse()

                            # path is the Eulerian cycle if connected. For holes, we should get a single Eulerian cycle that encompasses all edges (since all vertices have even degree).
                            # In orth polygon with holes, the boundary is formed by multiple disjoint cycles. But those would not share vertices if truly disjoint. 
                            # Actually, holes should appear as separate cycles not connected by edges. If they share vertices, we wouldn't have distinct polygons. 
                            # We must have done a BFS of entire component, so if holes are disconnected cycles, they'd form separate components. We handle them separately above.

                            # So for each connected component, we get exactly one Eulerian cycle covering all edges. But that cycle might represent multiple polygons if they are disconnected?
                            # Actually, if there's a hole, it forms a separate closed loop of edges not connected to the outer boundary by edges (they do not share edges, might share a vertex?). 
                            # Orth polygon holes typically do not share vertices with the outer boundary. They must be separate sets of edges, hence separate components.

                            # Conclusion: Each connected component is actually one closed polygon (possibly with a complex shape). If there are holes, they would be separate connected components in the boundary graph.

                            # Thus, this path is the single cycle we need.
                            return [path]

                        cycles = eulerian_cycles()

                        # For each cycle in cycles (though we expect one cycle per component), count sides:
                        for cycle_path in cycles:
                            # cycle_path is a list of vertices forming a cycle.
                            # Convert vertices to edges and determine directions
                            # Each consecutive pair in cycle_path forms an edge
                            # The last connects back to the first
                            directions = []
                            for k in range(len(cycle_path)):
                                v1 = cycle_path[k]
                                v2 = cycle_path[(k+1)%len(cycle_path)]
                                # direction: horizontal if v1.r == v2.r else vertical
                                (r1,c1) = v1
                                (r2,c2) = v2
                                if r1 == r2:
                                    dir_type = 'H'
                                else:
                                    dir_type = 'V'
                                directions.append(dir_type)

                            # Count sides: start with sides = 1, increment each time direction changes
                            sides = 1
                            for idx in range(1, len(directions)):
                                if directions[idx] != directions[idx-1]:
                                    sides += 1
                            # Check closing edge direction change:
                            # Actually we accounted for all edges in a cycle, the last direction is checked against the first one:
                            # The cycle is closed, so if directions[0] != directions[-1], that was counted already in the loop when idx=1 started from directions[1].

                            # Add this polygon's sides:
                            sides_count += sides

                # Now we have total sides_count for all polygons in this region.
                price = area * sides_count
                total_price += price

    end = time.perf_counter_ns()
    elapsed_ns = end - start
    elapsed_s = elapsed_ns / 1_000_000_000

    # Print total price and time with high precision
    print(total_price)
    print(f"{elapsed_s:.9f} s")

if __name__ == "__main__":
    main()

13732
0.012179100 s


In [5]:
from collections import defaultdict

region_set = set(region_cells)
area = len(region_cells)

# Collect boundary edges
edges = []
for (r,c) in region_cells:
    # Check up edge
    if r == 0 or grid[r-1][c] != plant_type:
        # horizontal top edge from (r,c) to (r,c+1)
        edges.append(((r,c),(r,c+1)))
    # Check down edge
    if r == rows-1 or grid[r+1][c] != plant_type:
        # horizontal bottom edge from (r+1,c) to (r+1,c+1)
        edges.append(((r+1,c),(r+1,c+1)))
    # Check left edge
    if c == 0 or grid[r][c-1] != plant_type:
        # vertical left edge from (r,c) to (r+1,c)
        edges.append(((r,c),(r+1,c)))
    # Check right edge
    if c == cols-1 or grid[r][c+1] != plant_type:
        # vertical right edge from (r,c+1) to (r+1,c+1)
        edges.append(((r,c+1),(r+1,c+1)))

# Build adjacency for boundary graph
adj = defaultdict(set)
for e in edges:
    v1, v2 = e
    adj[v1].add(v2)
    adj[v2].add(v1)

# Put all edges in a set of unused edges in a normalized form (smallest vertex first)
unused_edges = set()
for v1, v2 in edges:
    if v1 < v2:
        unused_edges.add((v1, v2))
    else:
        unused_edges.add((v2, v1))

def follow_cycle(start_edge):
    # Follow edges to form a loop
    # start_edge: (v1,v2) with v1 < v2
    current, nxt = start_edge
    cycle_vertices = [current, nxt]
    while True:
        # Remove used edge
        e = (current, nxt) if current < nxt else (nxt, current)
        unused_edges.remove(e)
        current = nxt
        # Find next edge from 'current' that is unused
        next_vertex = None
        for nb in adj[current]:
            e_candidate = (current, nb) if current < nb else (nb, current)
            if e_candidate in unused_edges:
                next_vertex = nb
                break
        if next_vertex is None:
            # No more unused edges from current, cycle is complete
            break
        nxt = next_vertex
        cycle_vertices.append(nxt)
    return cycle_vertices

sides_count = 0
while unused_edges:
    start = next(iter(unused_edges))
    cycle = follow_cycle(start)
    # Determine directions of edges in this cycle
    directions = []
    for k in range(len(cycle)):
        v1 = cycle[k]
        v2 = cycle[(k+1)%len(cycle)]
        (r1,c1) = v1
        (r2,c2) = v2
        if r1 == r2:
            dir_type = 'H'
        else:
            dir_type = 'V'
        directions.append(dir_type)

    # Count how many sides: start with 1, increment on direction changes
    sides = 1
    for idx in range(1, len(directions)):
        if directions[idx] != directions[idx-1]:
            sides += 1
    sides_count += sides

price = area * sides_count
total_price += price

NameError: name 'region_cells' is not defined

In [7]:
import time
from collections import defaultdict, deque

def main():
    start = time.perf_counter_ns()  # Ensure this is a direct assignment

    # Read the input grid
    grid = []
    with open("input.txt", "r") as f:
        for line in f:
            line = line.strip()
            if line:
                grid.append(list(line))

    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False]*cols for _ in range(rows)]

    def get_neighbors(r, c):
        # Return orthogonal neighbors
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    total_price = 0

    # Find regions using DFS
    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                plant_type = grid[i][j]
                stack = [(i,j)]
                visited[i][j] = True
                region_cells = []
                while stack:
                    r, c = stack.pop()
                    region_cells.append((r,c))
                    for nr, nc in get_neighbors(r,c):
                        if not visited[nr][nc] and grid[nr][nc] == plant_type:
                            visited[nr][nc] = True
                            stack.append((nr,nc))

                # Compute area
                area = len(region_cells)

                # Collect boundary edges
                edges = []
                for (r,c) in region_cells:
                    # Check up edge
                    if r == 0 or grid[r-1][c] != plant_type:
                        edges.append(((r,c),(r,c+1)))  # horizontal top
                    # Check down edge
                    if r == rows-1 or grid[r+1][c] != plant_type:
                        edges.append(((r+1,c),(r+1,c+1))) # horizontal bottom
                    # Check left edge
                    if c == 0 or grid[r][c-1] != plant_type:
                        edges.append(((r,c),(r+1,c))) # vertical left
                    # Check right edge
                    if c == cols-1 or grid[r][c+1] != plant_type:
                        edges.append(((r,c+1),(r+1,c+1))) # vertical right

                # Build adjacency for boundary graph
                adj = defaultdict(set)
                for e in edges:
                    v1, v2 = e
                    adj[v1].add(v2)
                    adj[v2].add(v1)

                # Put all edges in a set of unused edges in normalized form
                unused_edges = set()
                for v1, v2 in edges:
                    if v1 < v2:
                        unused_edges.add((v1, v2))
                    else:
                        unused_edges.add((v2, v1))

                def follow_cycle(start_edge):
                    # Follow edges to form a loop
                    (current, nxt) = start_edge
                    cycle_vertices = [current, nxt]
                    while True:
                        # Remove used edge
                        e = (current, nxt) if current < nxt else (nxt, current)
                        unused_edges.remove(e)
                        current = nxt
                        # Find next edge from 'current' that is unused
                        next_vertex = None
                        for nb in adj[current]:
                            e_candidate = (current, nb) if current < nb else (nb, current)
                            if e_candidate in unused_edges:
                                next_vertex = nb
                                break
                        if next_vertex is None:
                            # No more unused edges from current, cycle complete
                            break
                        nxt = next_vertex
                        cycle_vertices.append(nxt)
                    return cycle_vertices

                # Find all cycles (loops)
                sides_count = 0
                while unused_edges:
                    start = next(iter(unused_edges))
                    cycle = follow_cycle(start)
                    # Determine directions of edges in this cycle
                    directions = []
                    for k in range(len(cycle)):
                        v1 = cycle[k]
                        v2 = cycle[(k+1)%len(cycle)]
                        (r1,c1) = v1
                        (r2,c2) = v2
                        if r1 == r2:
                            dir_type = 'H'
                        else:
                            dir_type = 'V'
                        directions.append(dir_type)

                    # Count how many sides: start with 1, increment on direction changes
                    sides = 1
                    for idx in range(1, len(directions)):
                        if directions[idx] != directions[idx-1]:
                            sides += 1
                    sides_count += sides

                # Compute price for this region
                price = area * sides_count
                total_price += price

    end = time.perf_counter_ns()  # Ensure this is a direct assignment
    elapsed_ns = end - start
    elapsed_s = elapsed_ns / 1_000_000_000

    # Print total price and elapsed time
    print(total_price)
    print(f"{elapsed_s:.9f} s")

if __name__ == "__main__":
    main()

TypeError: unsupported operand type(s) for -: 'int' and 'tuple'

In [9]:
from collections import defaultdict, deque

def calculate_price(grid):
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False] * cols for _ in range(rows)]

    def neighbors(r, c):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    total_price = 0

    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                # Identify region
                plant_type = grid[i][j]
                queue = deque([(i, j)])
                visited[i][j] = True
                region_cells = []
                while queue:
                    r, c = queue.popleft()
                    region_cells.append((r, c))
                    for nr, nc in neighbors(r, c):
                        if not visited[nr][nc] and grid[nr][nc] == plant_type:
                            visited[nr][nc] = True
                            queue.append((nr, nc))

                # Compute area
                area = len(region_cells)

                # Calculate number of sides
                sides = 0
                for r, c in region_cells:
                    for nr, nc in neighbors(r, c):
                        if grid[nr][nc] != plant_type:
                            sides += 1

                # Compute price
                price = area * sides
                total_price += price

    return total_price

def main():
    # Load the input file
    with open("sample-input-for-part-002-number-003.txt", "r") as f:
        grid = [list(line.strip()) for line in f if line.strip()]

    # Calculate the total price
    result = calculate_price(grid)

    # Print the result
    print("Total Price:", result)

if __name__ == "__main__":
    main()

Total Price: 512


In [11]:
from collections import defaultdict, deque

def calculate_price(grid):
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False] * cols for _ in range(rows)]

    def neighbors(r, c):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    total_price = 0

    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                # Identify region
                plant_type = grid[i][j]
                queue = deque([(i, j)])
                visited[i][j] = True
                region_cells = []
                while queue:
                    r, c = queue.popleft()
                    region_cells.append((r, c))
                    for nr, nc in neighbors(r, c):
                        if not visited[nr][nc] and grid[nr][nc] == plant_type:
                            visited[nr][nc] = True
                            queue.append((nr, nc))

                # Compute area
                area = len(region_cells)

                # Calculate number of sides
                sides = 0
                for r, c in region_cells:
                    for nr, nc in neighbors(r, c):
                        if grid[nr][nc] != plant_type:
                            sides += 1

                # Debugging output for each region
                print(f"Region type: {plant_type}, Area: {area}, Sides: {sides}, Price: {area * sides}")

                # Compute price
                price = area * sides
                total_price += price

    return total_price

def main():
    # Load the input file
    with open("sample-input-for-part-002-number-003.txt", "r") as f:
        grid = [list(line.strip()) for line in f if line.strip()]

    # Calculate the total price
    result = calculate_price(grid)

    # Print the result
    print("Total Price:", result)

if __name__ == "__main__":
    main()

Region type: A, Area: 28, Sides: 16, Price: 448
Region type: B, Area: 4, Sides: 8, Price: 32
Region type: B, Area: 4, Sides: 8, Price: 32
Total Price: 512


In [13]:
from collections import defaultdict, deque

def calculate_price(grid):
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False] * cols for _ in range(rows)]

    def neighbors(r, c):
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    def count_sides(region_cells, plant_type):
        sides = 0
        for r, c in region_cells:
            for nr, nc in neighbors(r, c):
                if grid[nr][nc] != plant_type:
                    sides += 1
        return sides

    total_price = 0

    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                # Identify region
                plant_type = grid[i][j]
                queue = deque([(i, j)])
                visited[i][j] = True
                region_cells = []
                while queue:
                    r, c = queue.popleft()
                    region_cells.append((r, c))
                    for nr, nc in neighbors(r, c):
                        if not visited[nr][nc] and grid[nr][nc] == plant_type:
                            visited[nr][nc] = True
                            queue.append((nr, nc))

                # Compute area
                area = len(region_cells)

                # Calculate number of sides
                sides = count_sides(region_cells, plant_type)

                # Debugging output for each region
                print(f"Region type: {plant_type}, Area: {area}, Sides: {sides}, Price: {area * sides}")

                # Compute price
                price = area * sides
                total_price += price

    return total_price

def main():
    # Load the input file
    with open("sample-input-for-part-002-number-002.txt", "r") as f:
        grid = [list(line.strip()) for line in f if line.strip()]

    # Calculate the total price
    result = calculate_price(grid)

    # Print the result
    print("Total Price:", result)

if __name__ == "__main__":
    main()

Region type: A, Area: 28, Sides: 16, Price: 448
Region type: B, Area: 4, Sides: 8, Price: 32
Region type: B, Area: 4, Sides: 8, Price: 32
Total Price: 512


In [16]:
import sys
from collections import defaultdict, deque

def main():
    # Read input
    grid = []
    with open("sample-input-for-part-002-number-002.txt", "r") as f:
        for line in f:
            line = line.strip()
            if line:
                grid.append(list(line))
    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False]*cols for _ in range(rows)]

    def neighbors(r, c):
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    total_price = 0

    # For each region: find all plots of that region by DFS/BFS
    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                plant_type = grid[i][j]
                stack = [(i,j)]
                visited[i][j] = True
                region_cells = []
                while stack:
                    r, c = stack.pop()
                    region_cells.append((r,c))
                    for nr, nc in neighbors(r,c):
                        if not visited[nr][nc] and grid[nr][nc] == plant_type:
                            visited[nr][nc] = True
                            stack.append((nr,nc))

                area = len(region_cells)

                # Identify boundary edges of the region
                region_set = set(region_cells)
                edges = []
                for (r,c) in region_cells:
                    # Up edge
                    if r == 0 or grid[r-1][c] != plant_type:
                        edges.append(((r,c),(r,c+1)))  
                    # Down edge
                    if r == rows-1 or grid[r+1][c] != plant_type:
                        edges.append(((r+1,c),(r+1,c+1)))
                    # Left edge
                    if c == 0 or grid[r][c-1] != plant_type:
                        edges.append(((r,c),(r+1,c)))
                    # Right edge
                    if c == cols-1 or grid[r][c+1] != plant_type:
                        edges.append(((r,c+1),(r+1,c+1)))

                # Build adjacency for vertices
                adj = defaultdict(list)
                for e in edges:
                    v1, v2 = e
                    adj[v1].append(v2)
                    adj[v2].append(v1)

                # Find connected components of boundary graph
                visited_vertices = set()

                def eulerian_cycle(component_vertices):
                    # Hierholzer's algorithm to find Eulerian cycle
                    # Build local adjacency
                    local_adj = {v: adj[v][:] for v in component_vertices}
                    start_v = next(iter(component_vertices))
                    stack = [start_v]
                    path = []
                    while stack:
                        u = stack[-1]
                        if local_adj[u]:
                            w = local_adj[u].pop()
                            stack.append(w)
                        else:
                            path.append(stack.pop())
                    path.reverse()
                    # path is the Eulerian cycle (since this is a single component forming a closed loop)
                    return path

                # sides counting
                total_sides = 0

                # Find all connected components in the boundary graph
                to_visit = list(adj.keys())
                processed = set()

                for start_vertex in to_visit:
                    if start_vertex not in processed:
                        # BFS to find component
                        comp_queue = deque([start_vertex])
                        component_vertices = set()
                        while comp_queue:
                            v = comp_queue.popleft()
                            if v in component_vertices:
                                continue
                            component_vertices.add(v)
                            for w in adj[v]:
                                if w not in component_vertices:
                                    comp_queue.append(w)

                        processed.update(component_vertices)

                        # Get Eulerian cycle for this component
                        cycle_path = eulerian_cycle(component_vertices)

                        # Count sides in the polygon formed by cycle_path
                        # cycle_path is a closed loop of vertices
                        # Extract edges and directions
                        directions = []
                        for k in range(len(cycle_path)):
                            v1 = cycle_path[k]
                            v2 = cycle_path[(k+1)%len(cycle_path)]
                            (r1,c1) = v1
                            (r2,c2) = v2
                            if r1 == r2:
                                dir_type = 'H'  # horizontal edge
                            else:
                                dir_type = 'V'  # vertical edge
                            directions.append(dir_type)

                        # Count number of sides
                        # A side is defined as a maximal straight run of edges in the same direction.
                        sides = 1
                        for idx in range(1, len(directions)):
                            if directions[idx] != directions[idx-1]:
                                sides += 1

                        # Add this polygon's sides to the region's total sides
                        total_sides += sides

                # Compute price: area * total_sides
                price = area * total_sides
                total_price += price

    print(total_price)

if __name__ == "__main__":
    main()

464


In [17]:
import sys
from collections import deque, defaultdict

def main():
    # Read the input grid
    with open("sample-input-for-part-002-number-003.txt", "r") as f:
        grid = [list(line.strip()) for line in f if line.strip()]

    rows = len(grid)
    cols = len(grid[0]) if rows > 0 else 0

    visited = [[False]*cols for _ in range(rows)]

    def neighbors(r, c):
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols:
                yield nr, nc

    total_price = 0

    # Function to find all boundary edges of a region
    def get_region_boundary_edges(region_cells, plant_type):
        edges = set()
        rcells = set(region_cells)
        for (r,c) in region_cells:
            # Up edge
            if r == 0 or grid[r-1][c] != plant_type:
                edges.add(((r,c),(r,c+1)))  # top horizontal edge
            # Down edge
            if r == rows-1 or grid[r+1][c] != plant_type:
                edges.add(((r+1,c),(r+1,c+1))) # bottom horizontal edge
            # Left edge
            if c == 0 or grid[r][c-1] != plant_type:
                edges.add(((r,c),(r+1,c))) # left vertical edge
            # Right edge
            if c == cols-1 or grid[r][c+1] != plant_type:
                edges.add(((r,c+1),(r+1,c+1))) # right vertical edge
        return edges

    # A helper to reconstruct polygons from a set of edges.
    # Each edge is ((r1,c1),(r2,c2)) and is either vertical or horizontal.
    # We'll find closed loops (polygons) by traversing edges. Since this is a grid-aligned polygon,
    # from each vertex, there should be an even number of edges (2 or more), allowing a closed polygon.
    def extract_polygons(edges):
        # Build adjacency of vertices
        adj = defaultdict(list)
        for e in edges:
            v1, v2 = e
            adj[v1].append(v2)
            adj[v2].append(v1)

        polygons = []
        used_edges = set()

        # edges are undirected; store them in a normalized form for checking usage
        def edge_key(u,v):
            return (u,v) if u < v else (v,u)

        # Traverse until all edges are used
        for start_vertex in adj:
            for nxt in adj[start_vertex]:
                ek = edge_key(start_vertex, nxt)
                if ek not in used_edges:
                    # Found an unused edge, start polygon tracing
                    polygon = [start_vertex, nxt]
                    used_edges.add(ek)
                    current = nxt
                    prev = start_vertex
                    # Follow polygon
                    while current != start_vertex:
                        # current vertex connections
                        # find next vertex which is not prev
                        candidates = adj[current]
                        next_vertex = None
                        for cand in candidates:
                            if cand != prev:
                                # Check if edge used
                                cek = edge_key(current, cand)
                                if cek not in used_edges:
                                    next_vertex = cand
                                    break
                        if next_vertex is None:
                            # This should not happen for a closed polygon
                            break
                        polygon.append(next_vertex)
                        used_edges.add(edge_key(current, next_vertex))
                        prev, current = current, next_vertex
                    polygons.append(polygon)
        return polygons

    # Count sides from a polygon vertex list
    # polygon is a list of vertices [(r0,c0),(r1,c1),...,(r0,c0)]
    # Actually, when we end the polygon traversal, we have start vertex repeated at end.
    # Each edge direction: H if r1==r2 else V if c1==c2
    def count_sides(polygon):
        # The polygon list includes the start vertex at both beginning and end
        directions = []
        length = len(polygon)
        # The last vertex in polygon should be the same as the first to close the loop
        # Ensure loop closure
        if polygon[0] != polygon[-1]:
            polygon.append(polygon[0])
            length += 1

        for i in range(length - 1):
            (r1,c1) = polygon[i]
            (r2,c2) = polygon[i+1]
            if r1 == r2:
                directions.append('H')
            else:
                directions.append('V')

        # Count how many times direction changes
        # Each maximal run of the same direction is one side
        if not directions:
            return 0

        sides = 1
        for i in range(1, len(directions)):
            if directions[i] != directions[i-1]:
                sides += 1
        return sides

    # Main logic to find all regions and compute their price
    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                plant_type = grid[i][j]
                # Flood fill to get region
                q = deque([(i,j)])
                visited[i][j] = True
                region_cells = []
                while q:
                    r,c = q.popleft()
                    region_cells.append((r,c))
                    for nr,nc in neighbors(r,c):
                        if not visited[nr][nc] and grid[nr][nc] == plant_type:
                            visited[nr][nc] = True
                            q.append((nr,nc))

                area = len(region_cells)
                # Get boundary edges
                region_edges = get_region_boundary_edges(region_cells, plant_type)
                # Extract polygons
                polygons = extract_polygons(region_edges)
                # Count total sides = sum of sides of all polygons (outer + holes)
                total_sides = 0
                for poly in polygons:
                    total_sides += count_sides(poly)

                price = area * total_sides
                total_price += price

    print(total_price)

if __name__ == "__main__":
    main()

428
